In [1]:
! pip install tokenizer datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import time
import torchdynamo
import torch
from typing import List
from kernl.optimizer.dynamo_backend import dynamo_backend_ofi

In [3]:
model_name = "BaptisteDoyen/camembert-base-xnli"
nli_model = AutoModelForSequenceClassification.from_pretrained(model_name)
nli_model = nli_model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)
torchdynamo.config.cache_size_limit = 256

In [4]:
# https://huggingface.co/BaptisteDoyen/camembert-base-xnli
dataset = load_dataset('xnli', 'fr')

inputs: list[torch.Tensor] = list()

for index, content in enumerate(dataset['test']):
    premise, hypothesis, label = content.values()
    x = tokenizer(premise, hypothesis, return_tensors='pt', pad_to_multiple_of=8, padding=True, truncation=True)
    inputs.append(x)

Found cached dataset xnli (/home/geantvert/.cache/huggingface/datasets/xnli/fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
complete_time = 0
for index, x in enumerate(inputs):
    x = x.to("cuda")

    with torch.inference_mode(), torch.cuda.amp.autocast(enabled=True, dtype=torch.float16, cache_enabled=True):
        start = time.time()
        _ = nli_model(**x)
        torch.cuda.synchronize()
        complete_time += time.time() - start

print(f"{complete_time=}")

complete_time=50.011566400527954


In [6]:
from kernl.implementations.cuda_graph import cuda_graphs_wrapper

nli_model2 = AutoModelForSequenceClassification.from_pretrained(model_name)
nli_model2 = nli_model2.eval().cuda()


pool: (int, int) = torch.cuda.graph_pool_handle()
def compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    dynamo_backend_ofi(gm)
    return cuda_graphs_wrapper(gm, example_inputs, pool=pool)

def run(*args, **kwargs):
    with torchdynamo.optimize(compiler):
        return nli_model2(*args, **kwargs)


Below we do a warmup, it builds the triton kernels optimized for each size.
Moreover, we try to make the cuda graphs pool memory as big as possible to avoid having to rebuild it during the inference.

In [7]:
# warmup
shapes = [(1, w) for w in range(8, 128+8, 8)]
with torch.inference_mode(), torch.cuda.amp.autocast(enabled=True, dtype=torch.float16, cache_enabled=True):
    for _ in range(100):
        for s in shapes:
            # print(s)
            with torch.inference_mode(), torch.cuda.amp.autocast(enabled=True, dtype=torch.float16, cache_enabled=True):
                x = {'input_ids': torch.randint(1, 10000, s, device="cuda").to(torch.long),
                   'attention_mask': torch.ones(s, device="cuda", dtype=torch.long)}
                a = run(**x)
                torch.cuda.synchronize()
print("finish")

kernel_fma took 7.927716493606567 seconds
kernel_fma took 8.874297380447388 seconds
kernel_fma took 4.047192096710205 seconds
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the 

In [8]:
torch.cuda.empty_cache()
complete_time = 0
count = 0
for index, x in enumerate(inputs):
    x = x.to("cuda")
    if x["input_ids"].shape[1] > 128:
        print("long input, pass")
        continue
    assert x["input_ids"].shape[0] == 1
    assert x["input_ids"].shape[1] <= 128
    assert x["input_ids"].shape[1] % 8 == 0
    torch.cuda.synchronize()
    with torch.inference_mode(), torch.cuda.amp.autocast(enabled=True, dtype=torch.float16, cache_enabled=True):
        start = time.time()
        _ = run(**x)
        torch.cuda.synchronize()
        complete_time += time.time() - start
        if time.time() - start > 0.1:
            print(index, f"{time.time() - start:.3f}", x["input_ids"].shape)

    count += 1
    if count % 1000 == 0:
        print(f"{count=}")
print(f"{count=}")
print(f"{complete_time=:.3f}")

0 5.699 torch.Size([1, 40])
1 5.917 torch.Size([1, 48])
3 5.880 torch.Size([1, 80])
4 6.103 torch.Size([1, 88])
12 6.285 torch.Size([1, 56])
14 6.382 torch.Size([1, 64])
15 6.522 torch.Size([1, 24])
19 6.614 torch.Size([1, 72])
21 6.915 torch.Size([1, 32])
209 6.802 torch.Size([1, 16])
383 6.952 torch.Size([1, 96])
count=1000
count=2000
count=3000
3480 7.148 torch.Size([1, 104])
count=4000
4482 7.216 torch.Size([1, 112])
4483 7.429 torch.Size([1, 128])
4484 7.332 torch.Size([1, 120])
count=5000
count=5010
complete_time=106.256


From previous run:

0 5.699 torch.Size([1, 40])
1 5.917 torch.Size([1, 48])
3 5.880 torch.Size([1, 80])
4 6.103 torch.Size([1, 88])
12 6.285 torch.Size([1, 56])
14 6.382 torch.Size([1, 64])
15 6.522 torch.Size([1, 24])
19 6.614 torch.Size([1, 72])
21 6.915 torch.Size([1, 32])
209 6.802 torch.Size([1, 16])
383 6.952 torch.Size([1, 96])
count=1000
count=2000
count=3000
3480 7.148 torch.Size([1, 104])
count=4000
4482 7.216 torch.Size([1, 112])
4483 7.429 torch.Size([1, 128])
4484 7.332 torch.Size([1, 120])
count=5000
count=5010
complete_time=106.256


In [9]:
premise, hypothesis, _ = dataset['test'][209].values()
x = tokenizer(premise, hypothesis, return_tensors='pt', pad_to_multiple_of=8, padding=True, truncation=True)
print(x["input_ids"].shape)
print(premise, hypothesis)

torch.Size([1, 16])
Personne ne savait où ils étaient. Leur destination était secrète.


In [10]:
premise, hypothesis, _ = dataset['test'][383].values()
x = tokenizer(premise, hypothesis, return_tensors='pt', pad_to_multiple_of=8, padding=True, truncation=True)
print(x["input_ids"].shape)
print(premise, hypothesis)

torch.Size([1, 96])
Et je lui ai demandé, tu sais, est-ce que je pourrais le faire, euh, as-tu besoin que je reste et que je le fasse ce soir ou  est-ce que je peux le finir pour demain midi, si c'est d'accord. J'ai demandé au client s'il serait en colère si ça attendait jusqu'à 14 h car j'avais un rendez-vous ce soir.


In [11]:
premise, hypothesis, _ = dataset['test'][3480].values()
x = tokenizer(premise, hypothesis, return_tensors='pt', pad_to_multiple_of=8, padding=True, truncation=True)
print(x["input_ids"].shape)
print(premise, hypothesis)

torch.Size([1, 104])
Nous avons conscience qu'il nous aurait été difficile d'argumenter en faveur d'un coûteux changement de la stratégie de défense de la NORAD (agence de défense aérospatiale nord-américaine) afin de contrer le danger d'une attaque de pirates de l'air kamikazes, avant même qu'une telle menace ait jamais eu lieu. Il en coûte plus de cinq millions de dollars par jour pour améliorer la position de défense du NORAD.


In [12]:
premise, hypothesis, _ = dataset['test'][4482].values()
x = tokenizer(premise, hypothesis, return_tensors='pt', pad_to_multiple_of=8, padding=True, truncation=True)
print(x["input_ids"].shape)
print(premise, hypothesis)

torch.Size([1, 112])
L'ensemble de l'œuvre de Dowd, et en particulier la série de chroniques nommée Flytrap (attrape-mouche) qui lui a valu le Pulitzer, est jusqu'ici un des exemples les plus brillants de la propension des baby-boomers à l'auto-flagellation. Dowd n'a jamais écrit et n'écrira jamais quoi que ce soit qui puisse être interprété comme de l'auto-punition de baby-boomer.


In [13]:
premise, hypothesis, _ = dataset['test'][4483].values()
x = tokenizer(premise, hypothesis, return_tensors='pt', pad_to_multiple_of=8, padding=True, truncation=True)
print(x["input_ids"].shape)
print(premise, hypothesis)

torch.Size([1, 128])
L'ensemble de l'œuvre de Dowd, et en particulier la série de chroniques nommée Flytrap (attrape-mouche) qui lui a valu le Pulitzer, est jusqu'ici un des exemples les plus brillants de la propension des baby-boomers à l'auto-flagellation. Pour de brillants exemples d'auto-châtiment de la part des boomers, ne cherchez pas plus loin que les pièces Flytrap gagnantes du Pulitzer de Dowd, et, en fait, la plupart de son travail en tant que chroniqueur.


In [14]:
premise, hypothesis, _ = dataset['test'][4484].values()
x = tokenizer(premise, hypothesis, return_tensors='pt', pad_to_multiple_of=8, padding=True, truncation=True)
print(x["input_ids"].shape)
print(premise, hypothesis)

torch.Size([1, 120])
L'ensemble de l'œuvre de Dowd, et en particulier la série de chroniques nommée Flytrap (attrape-mouche) qui lui a valu le Pulitzer, est jusqu'ici un des exemples les plus brillants de la propension des baby-boomers à l'auto-flagellation. En plus de l'un des exemples les plus brillants de l'autocastigation des baby-boomers, le travail de Dowd comprend de nombreuses chroniques sur la poursuite des environnementalistes radicaux.


In [15]:
sizes = [len(tokenizer(d["premise"], d["hypothesis"], pad_to_multiple_of=8, padding=True, truncation=True)['input_ids']) for d in dataset['test']]
print(sum(sizes) / len(sizes))
print(max(sizes))
print(len([1 for s in sizes if s > 100]))

49.66706586826347
128
5
